# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic). Please, note, that you must not rely on any specific location for the dataset, hence, any code like

```python
titanic_train = pd.read_csv("<location>/train.csv")
```

will fail and your notebook won't be validated and graded. Inputs to the functions are described explicitly in each case, and that's the only thing you can rely on.

In [30]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [31]:
plt.rcParams["figure.figsize"] = (6,6)

In [32]:
import numpy as np
import pandas as pd

In [33]:
STUDENT = "Itay Koren"
ASSIGNMENT = 4
TEST = False

In [34]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

In [35]:
titanic_train = pd.read_csv('train.csv', index_col='PassengerId')
titanic_test = pd.read_csv('test.csv', index_col='PassengerId')


df = pd.concat([titanic_train, titanic_test])

In [36]:
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must **replace the corresponding age value in the dataframe with her actual age at the time** (25) and return the dataset. Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return a copy of the input dataframe, and not perform replacement in the original dataframe. Structure and indexing must be the same as in input.

In [37]:
def fix_age(df):
    """Fix age for Julia Florence Siegel."""
    df_c = df.copy()
    tw_index = df_c.loc[df_c.Name.str.contains('Mrs. Tyrell William')].index
    df_c.loc[tw_index, ['Age']] = 25
    return df_c

In [38]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return **series**, indexed with values from `Embarked`, according to `.value_counts()` method semantics:

```
S    <number of male passengers in 3-d class, embarked at S, 20<=Age<=30>
C    <...>
Q    <...>
Name: Embarked, dtype: int64
```

In [39]:
def embarked_stats(df):
    """Calculate embarkment port statistics."""
    return df[(df.Pclass == 3) & (df.Age >=20) & (df.Age <= 30) &(df.Sex == 'male')].Embarked.value_counts()

In [40]:
embarked_stats(df)

S    132
C     21
Q      7
Name: Embarked, dtype: int64

In [41]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age over all passengers, and fill missing age values in `Age` column.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [42]:
def fix_age(df):
    """Fix missing age values."""
    df_c = df.copy()
    avg_age = df_c.Age.mean()
    df_c['Age'] = df_c.Age.fillna(avg_age)
    return df_c
    

In [43]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's **only one child** like this.

In [44]:
def get_nursemaid(df):
    ticket = df[(df.Age < 10) & (df.Parch == 0)].Ticket.item()
    nurme_name, child_name  = df[(df.Ticket == ticket) ].Name.to_list()
    return (child_name, nurme_name)

In [45]:
get_nursemaid(df)

('Emanuel, Miss. Virginia Ethel', 'Dowdell, Miss. Elizabeth')

In [46]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **single string** with port letter.

In [72]:
(df[df.Age < 10].Embarked.value_counts() / df.Embarked.value_counts())

S    0.070022
C    0.051852
Q    0.032520
Name: Embarked, dtype: float64

In [73]:
def get_port(df):
    """Get port with the most children embarked."""
    s = (df[df.Age < 10].Embarked.value_counts() / df.Embarked.value_counts())
    return s.index[0]

In [160]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two values - average and maximum number of passengers per ticket.

In [161]:
df.groupby('Ticket')['Embarked'].value_counts().mean()
df.groupby('Ticket')['Embarked'].value_counts().max()
# df.groupby('Ticket')['Embarked'].value_counts()

11

In [162]:
df.groupby('Ticket')['Embarked'].value_counts()

Ticket       Embarked
110152       S           3
110413       S           3
110465       S           2
110469       S           1
110489       S           1
                        ..
W./C. 6608   S           5
W./C. 6609   S           1
W.E.P. 5734  S           2
W/C 14208    S           1
WE/P 5735    S           2
Name: Embarked, Length: 930, dtype: int64

In [163]:
df[df.Ticket == '110413']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
263,0.0,1,"Taussig, Mr. Emil",male,52.0,1,1,110413,79.65,E67,S
559,1.0,1,"Taussig, Mrs. Emil (Tillie Mandelbaum)",female,39.0,1,1,110413,79.65,E67,S
586,1.0,1,"Taussig, Miss. Ruth",female,18.0,0,2,110413,79.65,E68,S


In [164]:
ticket_gb = df.groupby('Ticket')['Embarked']

ticket_gb.value_counts().max()

11

In [165]:
def get_ticket_stats(df):
    """Calculate passenger per ticket statistics."""
    ticket_gb_vc = df.groupby('Ticket')['Embarked'].value_counts()
    return (ticket_gb_vc.mean(), ticket_gb_vc.max())

In [166]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate **fare per person for that ticket**, and then calculate averages for each class. Note, that you will need to apply `groupby` and you may consider using `.first()` of resulting `DataFrameGroupBy`. Also, caferully consider, in which order calculations are performed.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `pd.Series` with three elements, indexed by class:

```
1    <average per person fare in class 1>
2    <...>
3    <...>
Name: Pclass, dtype: float64
```

In [168]:
df.groupby(['Ticket']).Fare.mean()

Ticket
110152         86.500
110413         79.650
110465         52.000
110469         26.000
110489         26.550
                ...  
W./C. 6608     34.375
W./C. 6609      7.550
W.E.P. 5734    61.175
W/C 14208      10.500
WE/P 5735      71.000
Name: Fare, Length: 929, dtype: float64

In [ ]:
def get_fare_per_pass(df):
    """Calculate fare per passenger for different classes."""
    pass

In [ ]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with global average over all passengers. Now, you need to fill them **according to class and sex**. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [130]:
def fix_age_groupped(df):
    """Fill missing age values."""
    df_c = df.copy()
    gb_mean = df_c.groupby([df_c.Pclass.name, df_c.Sex.name])['Age'].mean()


    for c in [1,2,3]:
        for s in ['female','male']:
            idx = df_c[(df_c.Pclass == c) & (df_c.Sex == s) & (df_c['Age'].isna())].index
            df_c.loc[idx, ['Age']] = gb_mean.loc[c,s]

    return df_c

In [ ]:
fix_age_groupped

In [138]:
df.loc[(df.Pclass == 3) & (df.Sex == 'male'),['Age']].mean()

Age    25.962264
dtype: float64

In [139]:
df[df.Name == 'Spector, Mr. Woolf']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.05,NaN,S


In [137]:
d = fix_age_groupped(df)
d[d.Name == 'Spector, Mr. Woolf'] # expect to see 25.962


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1305,NaN,3,"Spector, Mr. Woolf",male,25.962264,0,0,A.5. 3236,8.05,NaN,S


In [ ]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of the number couples with partial survival or couples which died together, divided by total number of couples in class. If the survival status of one or both of spouses is not known, it must be considered as `0`.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column (see P7 as a reference).

In [ ]:
def find_couples(df):
    """Fill missing age values."""
    pass

In [ ]:
PROBLEM_ID = 9

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

In [ ]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")